# Evaluate sample designs

In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import datetime as dt
import shapely
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
import geopandas as gpd
import xarray as xr
import itertools
import scipy
import statsmodels.api as sm

from sklearn import cluster
from sklearn import neighbors
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate
from sklearn import metrics, cluster

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import ShuffleSplit, GroupKFold, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, lasso_path

from sklearn.preprocessing import KBinsDiscretizer, StandardScaler

import cvxpy as cp

/Users/etriesch/.pyenv/versions/3.9.5/envs/venv.ocean-carbon-sampling/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Simulation 1 - PCO2 unrelated to remote sensing features

In [6]:
MEAN_EFFECT = 15
SD_EFFECT = 60
sim_rs = np.random.normal(MEAN_EFFECT, SD_EFFECT, 200)
sim_mstd = pd.Series(sim_rs).agg(['mean', 'std'])

In [7]:
sim_rs_mstd

mean    13.105564
std     61.032304
dtype: float64

# Simulation 2 - pCO2 as a function of remote sensing features

# Simulation 3 - pCO2 as a function of remote sensing features and other